## Part 1: Preprocessing

In [61]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [3]:
#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [4]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [5]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [6]:
# Create a list of at least 10 column names to use as X data
features_list = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']


# Create X_df using your selected columns
X_df = attrition_df[features_list]

# Show the data types for X_df
X_df.dtypes


Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [7]:
X_df['OverTime'] = X_df['OverTime'].replace({'Yes': 1, 'No': 0})
X_df['OverTime'].value_counts()

/var/folders/h4/xm09cdqx717237dwyz2xxbbw0000gn/T/ipykernel_16738/3672064100.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_df['OverTime'] = X_df['OverTime'].replace({'Yes': 1, 'No': 0})


OverTime
0    1054
1     416
Name: count, dtype: int64

In [8]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)

In [9]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

### I dont know why we would do this here so I am going to do this before the split so that the test data also follows the same structure...
X_train['OverTime'].value_counts()

OverTime
0    798
1    304
Name: count, dtype: int64

In [10]:
X_train['OverTime'].dtypes

dtype('int64')

In [11]:
X_train_scaling = X_train.drop(columns='OverTime')
X_test_scaling = X_test.drop(columns='OverTime')

In [12]:
# Create a StandardScaler
X_scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler.fit(X_train_scaling)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train_scaling)
X_test_scaled = X_scaler.transform(X_test_scaling)

X_train_scaled_df = pd.DataFrame(X_train_scaled, columns = X_train_scaling.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns = X_test_scaling.columns)

In [13]:
X_train_post_scaled = pd.concat([X_train_scaled_df, X_train['OverTime'].reset_index()], axis=1)
X_test_post_scaled = pd.concat([X_test_scaled_df, X_test['OverTime'].reset_index()], axis=1)

In [14]:
X_train_post_scaled = X_train_post_scaled.set_index('index')
X_test_post_scaled = X_test_post_scaled.set_index('index')

In [15]:
display(X_train_post_scaled.head())
display(X_test_post_scaled.head())

,Education,Age,DistanceFromHome,JobSatisfaction,StockOptionLevel,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,NumCompaniesWorked,OverTime
index,,,,,,,,,,
464,0.080843,-0.000597,-1.009337,1.167247,-0.929636,1.790592,-0.142174,-0.350760,0.505016,0
512,1.070722,-1.097736,-0.761501,1.167247,-0.929636,0.356557,-0.483145,-0.350760,-1.093053,0
480,1.070722,-0.768595,0.353763,-1.589124,0.253342,1.790592,-0.994602,-0.663492,-0.693536,0
680,1.070722,-0.110311,-0.265828,1.167247,-0.929636,0.356557,-0.142174,-0.663492,-0.294019,0
1023,-0.909036,2.083967,-1.009337,-1.589124,0.253342,1.790592,-0.653631,-0.350760,-0.294019,0


,Education,Age,DistanceFromHome,JobSatisfaction,StockOptionLevel,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,NumCompaniesWorked,OverTime
index,,,,,,,,,,
1291,1.070722,-0.000597,0.105926,-1.589124,-0.929636,-2.511514,0.539768,-0.663492,-0.693536,0
1153,-0.909036,-2.085162,-0.761501,1.167247,-0.929636,1.790592,-1.165088,-0.663492,-0.693536,1
720,0.080843,-0.768595,1.592945,0.248457,-0.929636,0.356557,-0.312660,-0.663492,0.505016,1
763,1.070722,-0.329739,0.105926,0.248457,0.253342,0.356557,-0.994602,-0.663492,-0.693536,1
976,0.080843,2.083967,1.716863,-0.670333,0.253342,0.356557,2.074137,4.027495,0.505016,1


In [16]:
# Create a OneHotEncoder for the Department column
dept_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
dept_encoded = dept_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
dept_encoded_train = dept_encoder.transform(y_train[['Department']])
dept_encoded_test = dept_encoder.transform(y_test[['Department']])

dept_colums = dept_encoder.get_feature_names_out(['Department'])

dept_encoded_train_df = pd.DataFrame(dept_encoded_train, columns=dept_colums)
dept_encoded_test_df = pd.DataFrame(dept_encoded_test, columns=dept_colums)

In [17]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoded = attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
attrition_encoded_train = attrition_encoder.transform(y_train[['Attrition']])
attrition_encoded_test = attrition_encoder.transform(y_test[['Attrition']])

attrition_colums = attrition_encoder.get_feature_names_out(['Attrition'])

attrition_encoded_train_df = pd.DataFrame(attrition_encoded_train, columns=attrition_colums)
attrition_encoded_test_df = pd.DataFrame(attrition_encoded_test, columns=attrition_colums)

In [18]:
### COMBINING ALL Y encoded dfs

y_train_processed_df = pd.concat([
    dept_encoded_train_df,
    attrition_encoded_train_df],
    axis=1
    )

y_test_processed_df = pd.concat([
    dept_encoded_test_df,
    attrition_encoded_test_df],
    axis=1
    )

In [85]:
dept_encoded_train_df = np.array(dept_encoded_train_df)
attrition_encoded_train_df = np.array(attrition_encoded_train_df)

In [87]:
dept_encoded_train_df = dept_encoded_train_df.reshape(-1, 3)
attrition_encoded_train_df = attrition_encoded_train_df.reshape(-1, 2)

In [89]:
print("Reshaped Department target shape:", dept_encoded_train_df.shape)
print("Reshaped Attrition target shape:", attrition_encoded_train_df.shape)

Reshaped Department target shape: (1102, 3)
Reshaped Attrition target shape: (1102, 2)


## Create, Compile, and Train the Model

In [91]:
tf.keras.backend.clear_session()

In [93]:
# Find the number of columns in the X training data
num_features = X_train_post_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(num_features,), name='input_layer')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu', name='shared_layer1')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu', name='shared_layer2')(shared_layer1)

In [95]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_dense = layers.Dense(32, activation='relu', name='department_dense')(shared_layer2)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', name='department_output')(department_dense)

print("Expected department output shape:", department_output.shape)

Expected department output shape: (None, 3)


In [97]:
## Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_dense = layers.Dense(32, activation='relu', name = 'attrition_dense')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(2, activation='softmax', name='attrition_output')(attrition_dense)

print("Expected attrition output shape:", attrition_output.shape)

Expected attrition output shape: (None, 2)


In [109]:
# Create the model
model = Model(inputs=input_layer, outputs=[attrition_output, 
                                           department_output])

# Compile the model
model.compile(optimizer='adam', 
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'categorical_crossentropy'}, 
              metrics={'department_output': 'accuracy', 
                       'attrition_output': 'accuracy'})




# Summarize the model
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer1       │ (None, 64)        │        704 │ input_layer[0][0] │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer2       │ (None, 128)       │      8,320 │ shared_layer1[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_dense     │ (None, 32)        │      4,128 │ shared_layer2[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_dense    │ (None, 32)        │      4,128 │ shared_layer2[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ attrition_dense[… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ department_dense… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [111]:
print("Department output shape:", dept_encoded_train_df.shape[1])
print("Attrition output shape:", attrition_encoded_train_df.shape[1])

Department output shape: 3
Attrition output shape: 2


In [113]:
print("Department target shape:", dept_encoded_train_df.shape)
print("Attrition target shape:", attrition_encoded_train_df.shape)

Department target shape: (1102, 3)
Attrition target shape: (1102, 2)


In [115]:
print("Input data shape:", X_train_post_scaled.shape)
print("Department target shape:", dept_encoded_train_df.shape)  # Should be (num_samples, 3)
print("Attrition target shape:", attrition_encoded_train_df.shape)  # Should be (num_samples, 2)

Input data shape: (1102, 10)
Department target shape: (1102, 3)
Attrition target shape: (1102, 2)


In [120]:
# Train the model
model.fit(
    X_train_post_scaled,
    {
        'department_output': dept_encoded_train_df,
        'attrition_output': attrition_encoded_train_df,
    },
    epochs=100,  
    batch_size=32, 
    validation_split=0.2
)

Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8347 - attrition_output_loss: 0.4628 - department_output_accuracy: 0.6087 - department_output_loss: 0.8400 - loss: 1.3029 - val_attrition_output_accuracy: 0.8281 - val_attrition_output_loss: 0.4966 - val_department_output_accuracy: 0.6063 - val_department_output_loss: 0.8445 - val_loss: 1.3405
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8462 - attrition_output_loss: 0.4074 - department_output_accuracy: 0.6428 - department_output_loss: 0.8093 - loss: 1.2170 - val_attrition_output_accuracy: 0.8371 - val_attrition_output_loss: 0.4648 - val_department_output_accuracy: 0.6335 - val_department_output_loss: 0.8490 - val_loss: 1.3137
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8794 - attrition_output_loss: 0.3418 - department_output_accuracy: 0.6738 - department_output_loss: 0.7329 - loss: 1.0745 - val_attrition_output_accuracy: 0.8371 - 

In [126]:
# Evaluate the model with the testing data
results = model.evaluate(np.array(X_test), {
        'department_output': dept_encoded_test_df,
        'attrition_output': attrition_encoded_test_df
    })
results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.7898 - attrition_output_loss: 24.9922 - department_output_accuracy: 0.5397 - department_output_loss: 41.5114 - loss: 66.7904


[65.26966094970703,
 22.50933265686035,
 40.8958625793457,
 0.8152173757553101,
 0.5570651888847351]

In [57]:
# Print the accuracy for both department and attrition


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 